In [1]:
from transformers import AutoModelForCausalLM as AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from peft import *
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup

cache_dir='/var/tmp/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [11]:
model_name = 'bigscience/bloomz-560m' #'google-t5/t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = AutoModel.from_pretrained(model_name,cache_dir=cache_dir)
print(model)
print(model.config)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [17]:
# num_virtual_tokens important
peft_config = PrefixTuningConfig(task_type="CAUSAL_LM", num_virtual_tokens=20)
#peft_config = PromptEncoderConfig(task_type="CAUSAL_LM", num_virtual_tokens=20, encoder_hidden_size=128)
"""peft_config = PromptTuningConfig(task_type= TaskType.CAUSAL_LM ,
                                         tokenizer_name_or_path=model_name,
                                         prompt_tuning_init=PromptTuningInit.TEXT,
                                         prompt_tuning_init_text='classify:',
                                         num_virtual_tokens=8))
                                    
"""
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 983,040 || all params: 560,197,632 || trainable%: 0.1755


In [4]:
dataset_name = "twitter_complaints"
dataset = datasets.load_dataset("ought/raft", dataset_name,cache_dir=cache_dir)
print(dataset)
print(dataset["train"][0])
print(LABEL_TO_TEXT:=dataset["train"].features["Label"].names)

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})
{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}
['Unlabeled', 'complaint', 'no complaint']


In [5]:
x = dataset["train"][0]


In [6]:
def pp(x):
    tokenized_x = tokenizer( x['Tweet text'] ,add_special_tokens=True)#.to('cuda')
    tokenized_label = tokenizer( LABEL_TO_TEXT[x['Label']] )
    print(tokenized_x)

    model_input={}
    model_label={}
    model_input['input_ids'] = tokenized_x['input_ids'] + tokenized_label['input_ids']
    model_input['attention_mask'] = [1] * len(model_input['input_ids'] )
    model_input['labels'] = [-100] * (len(tokenized_x['input_ids'])) + tokenized_label['input_ids']
    
    return model_input

tokenized_train_dataset = dataset['train'].map(pp,remove_columns=dataset['train'].column_names)
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 50
})

In [7]:
tokenized_train_dataset[0]

{'input_ids': [35,
  169403,
  15296,
  36272,
  525,
  3928,
  1119,
  632,
  2670,
  3968,
  15270,
  1936,
  106863],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  1936,
  106863]}

In [8]:
dcwp = DataCollatorWithPadding(tokenizer=tokenizer,padding=True,max_length=512)
#dcwp = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False,)

dcwp(tokenized_train_dataset[0:4])

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
import evaluate
import numpy as np
def compute_metric(output):
    inp , label = output
    print(inp.shape,label.shape)
    #print(output)
    return {'acc':0}

batch_size = 1
trainer = Trainer(model,
        #data_collator=dcwp,
        train_dataset=tokenized_train_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metric,
        args=TrainingArguments(
                output_dir=output_dir,
                save_total_limit=1,
                num_train_epochs=50,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                
                
                
                
        ))

x = tokenized_train_dataset.select(range(1))
print(x[0])
print(len(x[0]['input_ids']))
print(len(x[0]['attention_mask']))
print(len(x[0]['labels']))

print(trainer.evaluate(x))
#trainer.train()

{'input_ids': [35, 169403, 15296, 36272, 525, 3928, 1119, 632, 2670, 3968, 15270, 1936, 106863], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1936, 106863]}
13
13
13


(1, 21, 250880) (1, 13)
{'eval_loss': 10.591363906860352, 'eval_acc': 0, 'eval_runtime': 0.0247, 'eval_samples_per_second': 40.554, 'eval_steps_per_second': 40.554}


Step,Training Loss
500,5.188400
1000,0.738900
1500,0.684100
2000,0.602600
2500,0.462800


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: F

TrainOutput(global_step=2500, training_loss=1.535362664794922, metrics={'train_runtime': 74.3842, 'train_samples_per_second': 33.609, 'train_steps_per_second': 33.609, 'total_flos': 117720865996800.0, 'train_loss': 1.535362664794922, 'epoch': 50.0})

In [18]:
model = model.to('cpu')
with torch.no_grad():
   print(dataset['train'][0])
   mi = tokenizer("Translate to English: Je t’aime.",return_tensors='pt')
   print(mi)
   o =  model.generate( input_ids=mi['input_ids'], max_new_tokens=10)#, attention_mask  = mi['attention_mask'])
   print(o,o.shape)
   
   o = tokenizer.decode(o[0])
   print(o)
   
   

{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}
{'input_ids': tensor([[153772,    427,   9522,   6395,  76721,  68258,     17]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 49152 but got size 1024 for tensor number 1 in the list.